<a href="https://colab.research.google.com/github/jeffheaton/present/blob/master/youtube/video/fft-frequency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright 2022 by [Jeff Heaton](https://www.heatonresearch.com/), released under [LGPLv3](https://www.gnu.org/licenses/lgpl-3.0.en.html)

[YouTube video about this code](https://www.youtube.com/watch?v=rj9NOiFLxWA)

In [4]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

PATH = "C:/Users/mason/Desktop/作業/科技系/教育大數據專題製作/EDU-Data/output/audio.wav"

! pip install -U kaleido

# 配置
FPS = 30
FFT_WINDOW_SECONDS = 0.25 # 音頻中每個 FFT 窗口的秒數

# 顯示的音符範圍
FREQ_MIN = 10
FREQ_MAX = 1000

# 要顯示的音符數量
TOP_NOTES = 3

# 音符的名稱
NOTE_NAMES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

# 輸出的尺寸。通常使用 SCALE 以獲得更高的分辨率，除非你需要非標準的長寬比。
RESOLUTION = (1920, 1080)
SCALE = 2 # 0.5=QHD(960x540), 1=HD(1920x1080), 2=4K(3840x2160)

Note: not using Google CoLab


In [5]:
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from scipy.io import wavfile # get the api
import os
#!pip install -U kaleido
# Get a WAV file from GDrive, such as:
# AUDIO_FILE = os.path.join(PATH,'short_popcorn.wav')

# Or download my sample audio
#!wget https://github.com/jeffheaton/present/raw/master/youtube/video/sample_audio/piano_c_major_scale.wav
AUDIO_FILE = PATH

fs, data = wavfile.read(os.path.join(PATH,AUDIO_FILE)) # load the data
print(fs)
audio = data.T[0] # this is a two channel soundtrack, get the first track
FRAME_STEP = (fs / FPS) # audio samples per video frame
FFT_WINDOW_SIZE = int(fs * FFT_WINDOW_SECONDS)
AUDIO_LENGTH = len(audio)/fs

16000


TypeError: object of type 'numpy.int16' has no len()

Several utility functions.

In [6]:
import plotly.graph_objects as go

def plot_fft(p, xf, fs, notes, dimensions=(960,540)):
  

  """
  繪製FFT（快速傅立葉變換）頻譜圖，並在圖上標註頂部音符。

  參數:
  p (numpy.ndarray): 幅值頻譜。
  xf (numpy.ndarray): 頻率值。
  fs (float): 取樣頻率。
  notes (list): 要在圖上標註的頂部音符列表。
  dimensions (tuple): 圖的尺寸（寬度，高度）。

  返回:
  go.Figure: Plotly 圖形對象。
  """

  layout = go.Layout(       # 圖形參數
    title="frequency spectrum",
    autosize=False,
    width=dimensions[0],
    height=dimensions[1],
    xaxis_title="Frequency (note)",
    yaxis_title="Magnitude",
    font={'size' : 24}
  )

  fig = go.Figure(layout=layout,          # go.Figure圖形形狀創建  layout設置圖形的布局參數
                  layout_xaxis_range=[FREQ_MIN,FREQ_MAX], #設圖形中xy軸範圍
                  layout_yaxis_range=[0,1]
                  )
  
  fig.add_trace(go.Scatter(     #這條折線圖代表了 FFT 後得到的頻譜數據  fig.add_trace加入圖形，go.Scatter繪製折線圖
    x = xf,
    y = p))
    #這行程式碼將一條折線圖添加到 Plotly 圖形中。x 軸的數據是 xf，即頻率值；y 軸的數據是 p，即幅值頻譜。

  
  for note in notes:        #在 Plotly 圖形中標註頂部音符的位置和內容
    fig.add_annotation(x=note[0]+10, y=note[2], #fig.add_annotation註釋添加到圖形中
            text=note[1],
            font = {'size' : 48},
            showarrow=False)
  return fig

def extract_sample(audio, frame_number):

  """
  提取給定幀數的音頻樣本。

  參數:
  audio (numpy.ndarray): 音頻數據。
  frame_number (int): 幀數。

  返回:
  numpy.ndarray: 提取的音頻樣本。
  """
  end = frame_number * FRAME_OFFSET   #開始位置
  begin = int(end - FFT_WINDOW_SIZE)  #結束位置

  if end == 0:
        # 沒有音頻數據，返回全零（開始）
    return np.zeros((np.abs(begin)),dtype=float)    
  elif begin<0:
        # 有一些音頻數據，填充零
    return np.concatenate([np.zeros((np.abs(begin)),dtype=float),audio[0:end]]) 
  else:
        # 通常情況，返回下一個樣本
    return audio[begin:end]    
  #這段程式碼確保了從音訊數據中提取的時間片段具有一定的長度並且在邊界條件下有所處理，從而確保了後續處理的準確性和穩定性。  
    

def freq_to_number(freq):
    """
    将频率转换为音符数字。
    
    参数:
    freq (float): 频率。
    
    返回:
    float: 音符数字。
    """
    if freq == 0:
        return float('-inf')  # 返回负无穷大，避免溢出错误
    if np.max(fft.real) < 0.001:  # 如果 FFT 頻譜中的最大實部值小於 0.001，則返回空列表 (音樂太短或音頻太高或低)
        return []  # 如果大於 0.001，則按照 fft.real 實部值排列大到小

    lst = [(x, y) for x, y in enumerate(fft.real)]  # 將排好的序列轉換為一個列表 (編號,實數值)
    # lst = [ (0,fft.real[0]), (1,fft.real[1]), .....]
    lst = sorted(lst, key=lambda x: x[1], reverse=True)  # 用 sorted 將 lst[] 按照指定的鍵 key，按照每個元素的第二個值(實部值)為依據進行排列，分析音樂信號中的頻率成分
    # lst[ (fft.real[0]), (fft.real[1]) ,....]
    idx = 0
    found = []  # 創建空列表[]
    found_note = set()  # 創建空集合{}   儲存已經找到的音符
    while idx < len(lst) and len(found) < num:  # 重複直到找到全部音符或全部跑完lst len<0 or idx>len
        f = lst[idx][0]  # f 頻率
        y = lst[idx][1]  # FFT 頻譜中對應的幅值
        n = freq_to_number(f)  # 計算頻率對應的音符數字
        n0 = int(round(n))  # round 四捨五入
        name = note_name(n0)  # 將數字轉為音符名稱

        if name not in found_note:  # 加入音符到集合
            found_note.add(name)
            s = [f, name, y]
            found.append(s)
        idx += 1

    return found  # 返回全部音符的種類數量

Run the FFT on individual samples of the audio and generate video frames of the frequency chart.

In [7]:
import numpy as np
import tqdm
#plot_fft動畫圖形，find_top_notes找音符，found頂部音符，extract_sample確保不發生錯誤

# 清理以 .png 結尾的文件
#!rm /content/*.png
#!pip install -U kaleido
#$pip install -U kaleido

# 轉換頻率到音符數字
def freq_to_number(f): 
  if f <= 0:
    return -1  # 或者返回其他特定值来表示无效的频率
  return 69 + 12*np.log2(f/440.0)

# 轉換音符數字到頻率
def number_to_freq(n): return 440 * 2.0**((n-69)/12.0)

# 獲取音符名稱
def note_name(n): return NOTE_NAMES[n % 12] + str(int(n/12 - 1))

# 漢寧窗口函數
window = 0.5 * (1 - np.cos(np.linspace(0, 2*np.pi, FFT_WINDOW_SIZE, False)))

# 計算 FFT 的頻率
xf = np.fft.rfftfreq(FFT_WINDOW_SIZE, 1/fs) #(實數值 生成動畫)
"""這行程式碼是用來產生傅立葉變換後的頻率軸
傅立葉變換通常返回實部和虛部，因此對於實數信號，通常只需考慮其中一半的頻譜，即正頻譜。
"""
FRAME_COUNT = int(AUDIO_LENGTH*FPS)
FRAME_OFFSET = int(len(audio)/FRAME_COUNT)

#(漢寧窗就像是給聲音穿上了一個保護罩，讓我們更清楚地聽到聲音的細節，減少其他聲音的干擾。)

def frequency_to_freq_domain_signal(top_notes_list, fs, N): #將頻率信息轉換為對應的頻域信號
    freq_domain_signal = np.zeros(N)  # 創建長度為 N 的頻域信號
    
    # 對於每個主要音符，設置頻域信號的相應位置為振幅或能量值
    for note in top_notes_list:
        frequency = note[0]  # 音符的頻率
        amplitude = note[2]  # 音符的振幅或能量值
        index = int(frequency * N / fs)  # 計算頻率對應的索引
        freq_domain_signal[index] = amplitude  # 將振幅或能量值設置到頻域信號中的相應位置
    
    return freq_domain_signal



# 主程式碼
mx = 0
top_notes_list = []  # 存储音乐中所有帧的主要音符


for frame_number in tqdm.tqdm(range(FRAME_COUNT)):
  
    #print("Current frame_number:", frame_number+1)  # 打印当前帧数
    # 提取音频样本
    sample = extract_sample(audio, frame_number)  # frame_number 是音频数据的帧数或时间点
    # 使用汉宁窗口函数进行加权(减少错误)
    #sample *= window
    # 使用快速傅里叶变换（FFT）计算加权后的音频样本的频谱
    fft = np.fft.rfft(sample*window)
    # 计算频谱的绝对值（取实部），得到频谱的振幅
    fft_abs = np.abs(fft).real
    # 更新最大振幅到 mx
    mx = max(np.max(fft_abs), mx)
    
    # 查找该帧的主要音符
    #top_notes = find_top_notes(fft_abs, TOP_NOTES)
    #top_notes_list.append(fft_abs)

    peaks, _ = find_peaks(fft_abs, height=0)  # 这里的 `find_peaks()` 函数需要根据你的具体情况选择

    # 从峰值中提取主要音符的频率
    main_frequencies = peaks * FRAME_COUNT / len(sample)  # 将峰值对应的索引转换为频率

    # 将频率转换为音符数字
    main_notes = [freq_to_number(frequency) for frequency in main_frequencies]

    top_notes_list.append(main_notes)


    
# 输出每个主要音符的信息
for frame_number, notes in enumerate(top_notes_list, start=1):
    for i, note in enumerate(notes, start=1):
        print(f"第 {frame_number} 帧的第 {i} 个主要音符: {note}")
#print("總共預期的偵數:", FRAME_COUNT)  # 打印预期的帧数


# 第二遍遍歷，生成動畫
for frame_number in tqdm.tqdm(range(FRAME_COUNT)):
    # 提取音頻樣本
    sample = extract_sample(audio, frame_number)
    # 使用漢寧窗口函數進行加權
    fft = np.fft.rfft(sample * window)
    # 使用快速傅立葉變換（FFT）計算加權後的音頻樣本的頻譜
    fft = np.abs(fft) / mx 
    # 查找頂部音符
    s = find_top_notes(fft,TOP_NOTES)   #find_top_notes從 FFT 頻譜中查找頂部音符，即具有最高幅值的音符
    # 繪製 FFT 圖譜並標註頂部音符
    fig = plot_fft(fft.real, xf, fs, s, RESOLUTION) #plot_fft繪製 FFT 圖譜並在圖上標註頂部音符
    # 將圖像保存為圖片文件
    !pip install -U kaleido
    import kaleido
    fig.write_image(f"/content/frame{frame_number}.png",scale=2)    #以便之後串成動畫  scale=2將圖片放大2倍存檔，增加解析度


#虽然理论上你可以将这两个循环合并为一个，
#但在实际中，这样做可能会导致每一帧都必须进行FFT计算两次，这会增加计算量并降低性能。
#因此，将这两个循环分开是一种更合理的做法，以便首先找出最大幅值，然后再生成动画。

#這段程式碼是用來製作一個音頻頻譜的動畫。
#首先，它會遍歷音頻文件的每個小部分（稱為幀），
#並計算每個幀的 FFT（快速傅立葉變換），這樣可以將幅值頻譜表示為頻率的函數。
#然後，它找到了所有幀中的最大幅值，以便將所有幀的頻譜進行縮放，使其適合於動畫中。
#接著，它再次遍歷每個幀，將 FFT 的結果轉換為頻譜圖，並在圖上標註出一些頂部的音符，
#最後將每個幀的圖像保存為一個圖像文件，以便後續的動畫製作。

NameError: name 'AUDIO_LENGTH' is not defined

In [7]:
!ffmpeg -y -r {FPS} -f image2 -s 1920x1080 -i frame%d.png -i {AUDIO_FILE} -c:v libx264 -pix_fmt yuv420p movie.mp4

  0%|          | 0/307 [00:00<?, ?it/s]

Use [ffmpeg](https://ffmpeg.org/) to combine the input audio WAV and the individual frame images into a MP4 video.

Download the generated movie.

In [9]:
from google.colab import files
files.download('movie.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>